# Notebook for creating the TfIdf Embeddings of the Story feature

## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Project/KickLearning/

/content/drive/.shortcut-targets-by-id/1iWelwZQ6yutjZacqvhb1xSPIvi_nlIiH/Project/KickLearning


Lybraries

In [3]:
from os.path import join
from os import listdir

import pandas as pd
import numpy as np

from feature_analysis.text_embeding import TextEncoder
# from feature_analysis.deep_text_embeding import TextEncoder as DeepTextEncoder

from time import time

In [5]:
data_path = join(".", "data", "texts")
destination_path = join(".", "data", "texts_embedded")

## Pooilng all the data together
make one single dataset

In [6]:
df_complete = pd.concat([pd.read_csv(join(data_path, file_name), usecols=["id", "story"]) for file_name in listdir(data_path)])

In [7]:
df_sample = df_complete.sample(frac=0.33)

## Embedding Data

In [9]:
encoder = TextEncoder(kpca_n_components=16)

In [10]:
start = time()
encoder.fit_tfidf_pipeline(data=df_sample.story.astype('U').to_list())
print(f"fitting time for TfIdf {(start-time())//60}:{(start-time())%60}")

fitting time for TfIdf -9.0:36.37919592857361


In [ ]:
embeddings = encoder.transform_pipeline(data=df_complete.story.astype('U').to_list(), fit_on_data=False)

## Saving Data

In [ ]:

df_complete.drop(columns="story", inplace=True)

df_complete = pd.concat([df_complete.id, pd.DataFrame(embeddings, index=df.index).add_prefix("text_")])

df_complete.to_csv(join(destination_path, "complete_story_embedding.csv"))

In [ ]:
df_complete.to_csv(join(destination_path, "complete_story_embedding.csv"))

In [ ]:
for story in df_complete.story.to_list()[:1000]:
    if type(story) is not str:
        print("found")
        x = story

found


# Old code

In [ ]:
for i, file_name in enumerate(listdir(data_path)):
    print(f"starting with document {i}")

    texts = pd.read_csv(join(data_path, file_name), usecols=["story"], nrows=100).story.to_list()

    encoder.fit_tfidf_pipeline(data=texts)

starting with document 0
starting with document 1
starting with document 2


In [ ]:
for i, file_name in enumerate(listdir(data_path)):
    print(f"starting with document {i}")

    df = pd.read_csv(join(data_path, file_name), usecols=["id", "story"], nrows=100)

    texts = df.story.to_list()

    embeddings = encoder.transform_pipeline(data=texts, fit_on_data=False)

    df.drop(columns="story", inplace=True)

    df = pd.concat([df.id, pd.DataFrame(embeddings, index=df.index).add_prefix("text_")])

    df.to_csv(join(destination_path, file_name))

starting with document 0
starting with document 1
starting with document 2


In [ ]:
encoder.new_data(data=["bene bene ciao, bello"])
encoder.tfidf_fit()

In [ ]:
encoder.new_data(data=["ciao dario ciao bello"])
encoder.tfidf_transform()

In [ ]:
embedding = encoder.get_data()


In [ ]:
embedding[0, 1]

0.0